## 模型简介

Langchain所封装的模型分为两类：
- 大语言模型 (LLM)
- 聊天模型 (Chat Models)

在后续的内容中，为简化描述，我们将使用 `LLM` 来指代大语言模型。

Langchain的支持众多模型供应商，包括OpenAI、ChatGLM、HuggingFace等。本教程中，我们将以OpenAI为例，后续内容中提到的模型默认为OpenAI提供的模型。

Langchain的封装，比如，对OpenAI模型的封装，实际上是指的是对OpenAI API的封装。

### LLM

`LLM` 是一种基于统计的机器学习模型，用于对文本数据进行建模和生成。LLM学习和捕捉文本数据中的语言模式、语法规则和语义关系，以生成连贯并合乎语言规则的文本。

在Langchain的环境中，LLM特指文本补全模型（text completion model）。

注，文本补全模型是一种基于语言模型的机器学习模型，根据上下文的语境和语言规律，自动推断出最有可能的下一个文本补全。

| 输入 | 输出 |
| -------- | ------- |
| 一条文本内容 | 一条文本内容 |

### 聊天模型 (Chat Models)

聊天模型是语言模型的一种变体。聊天模型使用语言模型，并提供基于"聊天消息"的接口。

| 输入 | 输出 |
| -------- | ------- |
| 一组聊天消息 | 一条聊天消息 |

`聊天消息` 除了消息内容文本，还会包含一些其他参数数据。这在后续的内容中会看到。

## Langchain与OpenAI模型

参考OpenAI [Model endpoint compatibility](https://platform.openai.com/docs/models/model-endpoint-compatibility) 文档，gpt模型都归为了聊天模型，而davinci, curie, babbage, ada模型都归为了文本补全模型。

| ENDPOINT | MODEL NAME |
| -------- | ------- |
| /v1/chat/completions | gpt-4, gpt-4-0613, gpt-4-32k, gpt-4-32k-0613, gpt-3.5-turbo, gpt-3.5-turbo-0613, gpt-3.5-turbo-16k, gpt-3.5-turbo-16k-0613 |
| /v1/completions | (Legacy)	text-davinci-003, text-davinci-002, text-davinci-001, text-curie-001, text-babbage-001, text-ada-001, davinci, curie, babbage, ada |

Langchain提供接口集成不同的模型。为了便于切换模型，Langchain将不同模型抽象为相同的接口 `BaseLanguageModel`，并提供 `predict` 和 `predict_messages` 函数来调用模型。

当使用LLM时推荐使用predict函数，当使用聊天模型时推荐使用predict_messages函数。

### 与LLM的交互

与LLM的交互，我们需要使用 `langchain.llms` 模块中的 `OpenAI` 类。

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")
response = llm.predict("是什么是AI?")
print(response)



AI（Artificial Intelligence）是指人工智能，是指计算机系统能够完成更加复杂的任务，而且能够像人一样进行思考和决策，并且能够自主地学习和学会新的知识。人工智能领域涉及模式识别、语音识别、自然语言处理、图像识别、机器学习、神经网络、机器人技术以及计算机视觉等


### 与聊天模型的交互

与聊天模型的交互，我们需要使用 `langchain.chat_models` 模块中的 `ChatOpenAI` 类。

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
response = chat.predict_messages([
    HumanMessage(content="什么是AI?")
])
print(response)

content='AI是人工智能（Artificial Intelligence）的缩写，指的是一种模拟人类智能的技术和系统。它通过模拟人类的思维和学习能力，使计算机能够执行类似于人类的智能任务，如语音识别、图像识别、自然语言处理、决策制定等。AI的目标是使计算机能够理解、学习和解决问题，从而提高工作效率和生活质量。AI技术包括机器学习、深度学习、自然语言处理、计算机视觉等。' additional_kwargs={} example=False


通过以下代码我们查看一下 `response` 变量的类型：

In [6]:
response.__class__

langchain.schema.messages.AIMessage

可以看到，它是一个 `AIMessage` 类型的对象。

接下来我们使用 `SystemMessage` 指令来指定模型的行为。如下代码指定模型对AI一无所知，在回答AI相关问题时，回答“我不知道”。

In [8]:
response = chat.predict_messages([
    SystemMessage(
        content="你是一个对人工智能一无所知的聊天机器人。当被问及人工智能时，你必须说“我不知道”"),
    HumanMessage(content="什么是深度学习?")
])
print(response)

content='我不知道。' additional_kwargs={} example=False


#### 3个消息类

Langchain框架提供了三个消息类，分别是 `AIMessage`、`HumanMessage` 和 `SystemMessage`。它们对应了OpenAI聊天模型API支持的不同角色 `assistant`、`user` 和 `system`。请参考 [OpenAI API文档 - Chat - Role](https://platform.openai.com/docs/api-reference/chat/create#chat/create-role)。

| Langchain类 | OpenAI角色 | 作用 |
| -------- | ------- | ------- |
| AIMessage | assistant | 模型回答的消息 |
| HumanMessage | user | 用户向模型的请求或提问 |
| SystemMessage | system | 系统指令，用于指定模型的行为 |